In [2]:
#text_file = "eng-ban.txt"
text_file = "ban-eng.txt"
# To open the text file we need to encode the text. Here, we use 'utf8' encoding
with open(text_file, encoding="utf8") as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    bangla, english = line.split(" >>> ")
    bangla = "[start] " + bangla + " [end]"
    text_pairs.append((english, bangla))

In [3]:
import random
for i in range(len(text_pairs)):
    print(random.choice(text_pairs));

('I know Bangla.', '[start] আমি বাংলা জানি. [end]')
('Bangla is my mother tounge.', '[start] বাংলা আমার মাতৃভাষা. [end]')
('Bangladesh is a south-asian country.', '[start] বাংলাদেশ দক্ষিন এশিয়ার একটি দেশ. [end]')
('Have you ever been to Bangladesh?', '[start] আপনি কি কখনো বাংলাদেশে গিয়েছেন? [end]')
('Sorry.', '[start] দুঃখিত. [end]')
('About seventy percentage of people are literate.', '[start] বাংলাদেশের প্রায় সত্তর ভাগ মানুষ স্বাক্ষর. [end]')
('It is a Bangla to English translation file.', '[start] এটা বাংলা থেকে ইংরেজি অনুবাদের নথি. [end]')
('I do not know him.', '[start] আমি উনাকে চিনি না. [end]')
('It is a Bangla to English translation file.', '[start] এটা বাংলা থেকে ইংরেজি অনুবাদের নথি. [end]')
('It is a Bangla to English translation file.', '[start] এটা বাংলা থেকে ইংরেজি অনুবাদের নথি. [end]')
('Sorry.', '[start] দুঃখিত. [end]')
('It is a Bangla to English translation file.', '[start] এটা বাংলা থেকে ইংরেজি অনুবাদের নথি. [end]')
('No.', '[start] না. [end]')
('I do not know him.', 

In [4]:
import random
random.shuffle(text_pairs)
total_data_length = len(text_pairs)
num_val_samples = int(0.15 * total_data_length)

In [5]:
num_train_samples = (total_data_length - (2 * num_val_samples))
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples]

In [6]:
import tensorflow as tf
import string
import re

strip_chars = string.punctuation + "?"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

In [7]:
f"[{re.escape(strip_chars)}]"

'[!"\\#\\$%\\&\'\\(\\)\\*\\+,\\-\\./:;<=>\\?@\\\\\\^_`\\{\\|\\}\\~\\?]'

In [7]:
# XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
from tensorflow.keras import layers

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return(tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", ""
    ))

vocab_size = 10000
sequence_length = 20
source_vectorization = layers.TextVectorization(
    max_tokens = vocab_size,
    output_mode = "int",
    output_sequence_length = sequence_length
)
target_vectorization = layers.TextVectorization(
    max_tokens = vocab_size,
    output_mode = "int",
    output_sequence_length = sequence_length + 1,
    standardize = custom_standardization
)

In [8]:
train_english_texts = [pair[0] for pair in train_pairs]
train_bangla_texts = [pair[1] for pair in train_pairs]
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_bangla_texts)

In [9]:
batch_size = 64

def format_dataset(eng, ban):
    eng = source_vectorization(eng)
    ban = source_vectorization(ban)
    return({"english": eng, "bangla": ban[:, :-1]}, ban[:, 1:])

def make_dataset(pairs):
    eng_texts, ban_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    ban_texts = list(ban_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, ban_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls = 4)
    return(dataset.shuffle(2048).prefetch(16).cache())

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [22]:
#print(list(train_ds.as_numpy_iterator()));

In [10]:
for inputs, targets in train_ds.take(1):
    print(f"inputs['english'].shape: {inputs['english'].shape}")
    print(f"inputs['bangla'].shape: {inputs['bangla'].shape}")
    print(f"target.shape: {targets.shape}")

inputs['english'].shape: (12, 20)
inputs['bangla'].shape: (12, 19)
target.shape: (12, 19)


In [11]:
from tensorflow import keras

embed_dim = 256
latent_dim = 1024

In [12]:
source = keras.Input(shape=(None,), dtype="int64", name="english")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(source)
encoded_source = layers.Bidirectional(layers.GRU(latent_dim), merge_mode="sum")(x)

In [13]:
past_target = keras.Input(shape=(None,), dtype="int64", name="bangla")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(past_target)
decoder_gru = layers.GRU(latent_dim, return_sequences=True)
x = decoder_gru(x, initial_state=encoded_source)
x = layers.Dropout(0.5)(x)
target_next_step = layers.Dense(vocab_size, activation="softmax")(x)
seq2seq_rnn = keras.Model([source, past_target], target_next_step)

In [29]:
#seq2seq_rnn.summary()

In [14]:
seq2seq_rnn.compile(
    optimizer = "rmsprop",
    loss = "sparse_categorical_crossentropy",
    metrics = ["accuracy"]
)
seq2seq_rnn.fit(train_ds, epochs=50, validation_data=val_ds)

Epoch 1/50
1/1 [==============================] - 22s 22s/step - loss: 2.9095 - accuracy: 0.0000e+00 - val_loss: 2.1079 - val_accuracy: 0.7778
Epoch 2/50
1/1 [==============================] - 2s 2s/step - loss: 2.7871 - accuracy: 0.8333 - val_loss: 1.5239 - val_accuracy: 0.7778
Epoch 3/50
1/1 [==============================] - 2s 2s/step - loss: 1.3435 - accuracy: 0.8333 - val_loss: 0.5896 - val_accuracy: 0.7778
Epoch 4/50
1/1 [==============================] - 2s 2s/step - loss: 0.3356 - accuracy: 0.8333 - val_loss: 0.6209 - val_accuracy: 0.4444
Epoch 5/50
1/1 [==============================] - 2s 2s/step - loss: 0.4784 - accuracy: 0.2778 - val_loss: 0.6181 - val_accuracy: 0.7778
Epoch 6/50
1/1 [==============================] - 2s 2s/step - loss: 0.3400 - accuracy: 0.8333 - val_loss: 0.5532 - val_accuracy: 0.7778
Epoch 7/50
1/1 [==============================] - 2s 2s/step - loss: 0.2449 - accuracy: 0.8333 - val_loss: 0.5153 - val_accuracy: 0.7778
Epoch 8/50
1/1 [===================